In [ ]:
import time
import threading
import queue
import speech_recognition as sr
from transformers import pipeline

class SpeechToTextThread(threading.Thread):
    def __init__(self, language, queue):
        super().__init__()
        self.language = language
        self.queue = queue
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()

    def run(self):
        print("Speech-to-Text thread started.")
        while True:
            with self.microphone as source:
                print("Listening...")
                audio = self.recognizer.listen(source)

            try:
                text = self.recognizer.recognize_google(audio, language=self.language)
                self.queue.put(text)
            except sr.UnknownValueError:
                print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print("Could not request results from Google Speech Recognition service; {0}".format(e))
            except Exception as e:
                print("Error:", e)

class TranslationThread(threading.Thread):
    def __init__(self, source_language, queue):
        super().__init__()
        self.source_language = source_language
        self.queue = queue
        self.translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en")

    def run(self):
        print("Translation thread started.")
        while True:
            text = self.queue.get()
            translation = self.translator(text, src=self.source_language, tgt="en")[0]["translation_text"]
            print("Translation:", translation)

def main():
    language = "en-US"  # Change it according to your source language
    q = queue.Queue()

    # Start speech-to-text thread
    stt_thread = SpeechToTextThread(language, q)
    stt_thread.daemon = True
    stt_thread.start()

    # Start translation thread
    translation_thread = TranslationThread(language, q)
    translation_thread.daemon = True
    translation_thread.start()

    # Keep the main thread alive
    while True:
        time.sleep(1)

if __name__ == "__main__":
    main()
